**Import Libraries**

In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

#!pip install folium
import folium

#!pip install geocoder 
import geocoder

#!pip install geopy
from geopy.geocoders import Nominatim




**Get Request and Preporecessing**

In [15]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'html')
table = soup.find_all('table')[0] 
df_ = pd.read_html(str(table))
df = df_[0]
df.columns = df.iloc[0]
df.drop([0,1,2], inplace = True)

df.rename({"Postcode": "PostalCode"}, axis=1, inplace=True)
df = df[df["Borough"] != "Not assigned"]
df[df["Neighbourhood"] == "Not assigned"]
df[df["Borough"] == "Queen's Park"]
df_agg = df.groupby(["PostalCode", "Borough"]).agg({'Neighbourhood': lambda x: ', '.join(x)}).reset_index()
df[df["PostalCode"] == "M5B"]
df_agg[df_agg["PostalCode"] == "M5B"]
df_geo = pd.read_csv("http://cocl.us/Geospatial_data")
df_geo.rename({"Postal Code": "PostalCode"}, axis=1, inplace=True)
df_agg_geo = pd.merge(df_agg,df_geo,on="PostalCode",how="left")

In [16]:
df.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor


In [17]:
df_agg_geo.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


**Toronto, Canada Coordinates**

In [19]:
address = 'Toronto, Canada'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

C:\Users\ky55884\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


The geograpical coordinate of the City of Toronto are 43.653963, -79.387207.


In [20]:
df_toronto = df_agg_geo[df_agg_geo['Borough'].str.contains('Toronto')].reset_index(drop=True)

**Clustering and Folium**

In [21]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=12)

X = df_toronto['Latitude']
Y = df_toronto['Longitude']
Z = np.stack((X, Y), axis=1)

kmeans = KMeans(n_clusters=4, random_state=0).fit(Z)

clusters = kmeans.labels_
colors = ['red', 'green', 'blue', 'yellow']
df_toronto['Cluster'] = clusters

for latitude, longitude, borough, cluster in zip(df_toronto['Latitude'], 
                                                 df_toronto['Longitude'], 
                                                 df_toronto['Borough'], 
                                                 df_toronto['Cluster']):
    label = folium.Popup(borough, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(toronto_map)  

toronto_map